# Requirements

In [ ]:
'''
REQUIREMENTS BEFORE RUNNING THIS NOTEBOOK

Complete all of the following in GIS:
1.  Split OSM building polygons at vertices --> here called the "splitlines" layer.
2.  Calculate geometry attributes for the splitlines, adding: (i) central point-x, (ii) central point-y, and (iii) line bearing.
    --> write into columns (i) "Center_Lon", (ii) "Center_Lat", and (iii) "LINE_BEARING".
3.  Create a point feature class from "Center_Lon" and "Center_Lat".
4.  Calculate near distances from line central x,y point to nearest OSM road segment and associated angle from central point to road segment; in ArgGIS use "Near" tool
    --> write into (i) "osmRoads_NEAR_DIST" for distance, (ii) "osm_Roads_NEAR_ANGLE" for the angle, and (iii) and "osmRoads_NEAR_FID" for the FID identifier in the OSM roads layer.
5. Export GIS table to Excel and import into Python script in cell below.

'''

# Imports

In [ ]:
# Only required if using Google Colab and Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import shutil
import random
from glob import glob
import pyproj
gd = pyproj.Geod(ellps='WGS84')
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)

# Load dataframe

In [ ]:
DATA_FOLDER = # ADD PATH TO DATA e.g. '/content/drive/MyDrive/Project_Name/data/'

In [ ]:
splitlines_df = pd.read_excel(DATA_FOLDER+'splitlines_to_be_prepared.xlsx')
splitlines_df

,OBJECTID,osm_id,code,fclass,name,type,LINE_LENGTH,LINE_BEARING,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,osmRoads_NEAR_ANGLE,osmRoads_NEAR_DIST,osmRoads_NEAR_FID
0,1,83471230,1500,building,,apartments,62.646302,182.411524,7.706395,45.096729,7.706361,45.096166,7.706378,45.096448,-86.363996,20.390748,1202
1,2,83471230,1500,building,,apartments,12.109261,272.427701,7.706361,45.096166,7.706208,45.096171,7.706285,45.096168,-79.177687,15.041523,1202
2,3,83471230,1500,building,,apartments,62.635242,2.411950,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,-86.364105,8.284624,1202
3,4,83471230,1500,building,,apartments,12.108680,92.375219,7.706241,45.096734,7.706395,45.096729,7.706318,45.096732,30.767259,6.583651,96
4,5,83471233,1500,building,,apartments,56.362096,182.904191,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,92.202407,7.641764,2080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10216,10217,1214584772,1500,building,,residential,17.299046,110.479065,7.698073,45.106321,7.698279,45.106267,7.698176,45.106294,21.066739,6.364065,111
10217,10218,1233637785,1500,building,,,11.122694,262.512272,7.655736,45.134323,7.655596,45.134310,7.655666,45.134316,81.772403,44.343549,3026
10218,10219,1233637785,1500,building,,,7.779111,352.529588,7.655596,45.134310,7.655583,45.134379,7.655589,45.134344,-87.541932,40.459978,464
10219,10220,1233637785,1500,building,,,11.122699,82.511550,7.655583,45.134379,7.655723,45.134392,7.655653,45.134386,85.625732,44.459302,3026


# Define functions

In [ ]:
def format_df_angles(df):
  """
  Formats an input dataframe to first correct for negative angles, then provide 'angle_diff', the difference
  in 'LINE_BEARING' of a splitline segment and the ange of the nearest OSM road.

  Arguments:
  df       -- pd.DataFrame; must contain columns "LINE_BEARING" and "osmRoads_NEAR_ANGLE"

  Returns:
  df       -- pd.DataFrame; with added columns "osmRoads_NEAR_ANGLE_corrected" and "angle_diff"
  """

  df['osm_Roads_NEAR_ANGLE_corrected'] = df.apply(lambda row: (row['osmRoads_NEAR_ANGLE'] + 360) if (row['osmRoads_NEAR_ANGLE']< 0) else row['osmRoads_NEAR_ANGLE'], axis=1)
  df['angle_diff'] = df['osm_Roads_NEAR_ANGLE_corrected'] - df['LINE_BEARING']
  df['angle_diff'] = df.apply(lambda row: row['angle_diff'] + 360 if row['angle_diff']<0 else row['angle_diff'], axis=1)
  return df

def nearest_to_road_segment(osm_id, additional_run, ang_thresh = 20, multiplier = 1.0):
  """
  Analyzes each line segment from each OSM building polygon to determine whether it is street-facing.

  Arguments:
  osm_id    --  int; a uniuqe identifier for each building polygon in the OSM database.
  addtl_run --  boolean; indicate False if it is the first time running the function, or True
                if running a second or subsequent time. This allow all columns with different
                multiplier values to be saved in the dataframe prepped_splitlines.
  ang_thresh -- int; a value in degrees representing a threshold to be added/subtracted from 90
                270 degrees to determine whether building line segments are street-facing. A street-
                facing line perfectly parallel to the street has an osm_Roads_NEAR_ANGLE_corrected value of 90 or 270.
                Thus this threshold allows for buildings not perfectly parallel to roads.
  multiplier -- float; a value typically ranging 1.0 to 1.5, multiplies the minimum distance
                to the nearest road segment. Useful when there are buildings that have
                multiple line segments facing the same road segment, e.g. where a building has
                one or more slight bends or jogs on the street-facing facade.
  Returns:
  return_df --  pd.DataFrame; contains all original rows associated with the osm_id, but adding a column 'st_facing_'
                indicating which segments are street-facing given the input multiplier. Results: 1 is a street-facing
                segment, and both 0 and NaN are not. A result of NaN means the segment was not within 90 or 270 degrees
                plus/minus the threshold.
  """

  filtered_df_lst = []
  # If the function is being run two or more times, set additional_run to True
  # and use the dataframe prepped_splitlines instead of splitlines_df.
  if additional_run == False:
    tmp_df = splitlines_df.loc[splitlines_df['osm_id']==osm_id].copy()
  else:
    tmp_df = prepped_splitlines.loc[prepped_splitlines['osm_id']==osm_id].copy()

  # The road_ids correspond to 'OBJECTID' numbers in GIS
  road_ids = tmp_df['osmRoads_NEAR_FID'].unique().tolist()
  # Create column label based on multiplier input, remove '.' because this character doesn't work in GIS tables
  multiplier_str = 'mult_'+str(multiplier).split('.')[0]+'pt'+str(multiplier).split('.')[1]

  # Keep rows in tmp_df only if the 'angle_diff' is 90 or 270 degrees plus/minus the angle threshold
  filtered_df = tmp_df.loc[(tmp_df['angle_diff']>90-ang_thresh) & (tmp_df['angle_diff']<90+ang_thresh) | (tmp_df['angle_diff']>270-ang_thresh) & (tmp_df['angle_diff']<270+ang_thresh)]

  if road_ids == []:
    return pd.DataFrame()
  # Iterate through road_ids and determine closest building line segment(s) for each based on minimum distance of all lines to the road segment.
  else:
    for road_id in road_ids:
      filtered2_df = filtered_df.loc[tmp_df['osmRoads_NEAR_FID']==road_id].copy()
      min_dist = filtered2_df['osmRoads_NEAR_DIST'].min()
      # Use the multiplier as a factor to increase min_dist
      min_dist = min_dist * multiplier
      filtered2_df['st_facing_' + multiplier_str] = filtered2_df.apply(lambda row: 1 if row['osmRoads_NEAR_DIST']<=min_dist else 0, axis=1)
      filtered_df_lst.append(filtered2_df)
      concat_df = pd.concat(filtered_df_lst)
    return_df = pd.merge(tmp_df, concat_df[['OBJECTID', 'st_facing_' + multiplier_str]], on='OBJECTID', how='left')
    return return_df

# Call functions to modify input data

In [ ]:
# Run format_df_angles to calculate the angle difference between osmRoads_NEAR_ANGLE
# and the LINE_BEARING of the splitline segments.

splitlines_df = format_df_angles(splitlines_df)
splitlines_df

,OBJECTID,osm_id,code,fclass,name,type,LINE_LENGTH,LINE_BEARING,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,osmRoads_NEAR_ANGLE,osmRoads_NEAR_DIST,osmRoads_NEAR_FID,osm_Roads_NEAR_ANGLE_corrected,angle_diff
0,1,83471230,1500,building,,apartments,62.646302,182.411524,7.706395,45.096729,7.706361,45.096166,7.706378,45.096448,-86.363996,20.390748,1202,273.636004,91.224480
1,2,83471230,1500,building,,apartments,12.109261,272.427701,7.706361,45.096166,7.706208,45.096171,7.706285,45.096168,-79.177687,15.041523,1202,280.822313,8.394612
2,3,83471230,1500,building,,apartments,62.635242,2.411950,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,-86.364105,8.284624,1202,273.635895,271.223945
3,4,83471230,1500,building,,apartments,12.108680,92.375219,7.706241,45.096734,7.706395,45.096729,7.706318,45.096732,30.767259,6.583651,96,30.767259,298.392040
4,5,83471233,1500,building,,apartments,56.362096,182.904191,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,92.202407,7.641764,2080,92.202407,269.298216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10216,10217,1214584772,1500,building,,residential,17.299046,110.479065,7.698073,45.106321,7.698279,45.106267,7.698176,45.106294,21.066739,6.364065,111,21.066739,270.587673
10217,10218,1233637785,1500,building,,,11.122694,262.512272,7.655736,45.134323,7.655596,45.134310,7.655666,45.134316,81.772403,44.343549,3026,81.772403,179.260131
10218,10219,1233637785,1500,building,,,7.779111,352.529588,7.655596,45.134310,7.655583,45.134379,7.655589,45.134344,-87.541932,40.459978,464,272.458068,279.928481
10219,10220,1233637785,1500,building,,,11.122699,82.511550,7.655583,45.134379,7.655723,45.134392,7.655653,45.134386,85.625732,44.459302,3026,85.625732,3.114182


In [ ]:
# Test one example on nearest_to_road_segment.

nearest_to_road_segment(808447137, additional_run=False, ang_thresh = 20, multiplier=1.0)

,OBJECTID,osm_id,code,fclass,name,type,LINE_LENGTH,LINE_BEARING,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,osmRoads_NEAR_ANGLE,osmRoads_NEAR_DIST,osmRoads_NEAR_FID,osm_Roads_NEAR_ANGLE_corrected,angle_diff,st_facing_mult_1pt0
0,3632,808447137,1500,building,,apartments,16.917763,72.553986,7.690499,45.095887,7.690705,45.095933,7.690602,45.095910,-71.415735,12.488689,199,288.584265,216.030279,NaN
1,3633,808447137,1500,building,,apartments,4.634557,114.279095,7.690705,45.095933,7.690758,45.095916,7.690731,45.095924,-157.213434,18.826192,203,202.786566,88.507471,0.0
2,3634,808447137,1500,building,,apartments,16.320838,117.706204,7.690758,45.095916,7.690942,45.095847,7.690850,45.095882,-157.213350,18.080129,203,202.786650,85.080446,0.0
3,3635,808447137,1500,building,,apartments,11.213298,112.683951,7.690942,45.095847,7.691073,45.095809,7.691008,45.095828,-157.213239,17.408322,203,202.786761,90.102810,0.0
4,3636,808447137,1500,building,,apartments,13.053357,204.287879,7.691073,45.095809,7.691005,45.095702,7.691039,45.095755,-157.213216,10.900846,203,202.786784,358.498904,NaN
5,3637,808447137,1500,building,,apartments,23.173876,295.035534,7.691005,45.095702,7.690738,45.095790,7.690872,45.095746,-157.213335,4.816392,203,202.786665,267.751131,1.0
6,3638,808447137,1500,building,,apartments,14.220195,294.099040,7.690738,45.095790,7.690573,45.095842,7.690656,45.095816,-157.213487,5.410174,203,202.786513,268.687473,0.0
7,3639,808447137,1500,building,,apartments,7.688366,311.108804,7.690573,45.095842,7.690499,45.095887,7.690536,45.095864,-157.213572,6.766573,203,202.786428,251.677625,0.0


In [ ]:
# For loop to run nearest_to_road_segment function to identify building line segments
# that are closest to each OSM road segment, and therefore are likely the street-facing segments.
# The recommended range for MULTIPLIER is 1.0 to 1.5; one can run this loop multiple times, thus creating
# different columns to visualize in GIS and thus select the best multiplier for the case study.

# ***If running the function more than once, afte the first run, set ADDITIONAL_RUN to True.***

MULTIPLIER = 1.0
ADDITIONAL_RUN = False

counter = 0
df_lst = []
for osm_id in splitlines_df['osm_id'].unique().tolist():
  temp_df = nearest_to_road_segment(osm_id, additional_run=ADDITIONAL_RUN, multiplier = MULTIPLIER)
  if not temp_df.empty:
    df_lst.append(temp_df)
  counter += 1
  if counter%500 == 0:
    print(counter)
print(counter)

prepped_splitlines = pd.concat(df_lst)
prepped_splitlines

500
1000
1500
1818


,OBJECTID,osm_id,code,fclass,name,type,LINE_LENGTH,LINE_BEARING,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,osmRoads_NEAR_ANGLE,osmRoads_NEAR_DIST,osmRoads_NEAR_FID,osm_Roads_NEAR_ANGLE_corrected,angle_diff,st_facing_mult_1pt0
0,1,83471230,1500,building,,apartments,62.646302,182.411524,7.706395,45.096729,7.706361,45.096166,7.706378,45.096448,-86.363996,20.390748,1202,273.636004,91.224480,0.0
1,2,83471230,1500,building,,apartments,12.109261,272.427701,7.706361,45.096166,7.706208,45.096171,7.706285,45.096168,-79.177687,15.041523,1202,280.822313,8.394612,NaN
2,3,83471230,1500,building,,apartments,62.635242,2.411950,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,-86.364105,8.284624,1202,273.635895,271.223945,1.0
3,4,83471230,1500,building,,apartments,12.108680,92.375219,7.706241,45.096734,7.706395,45.096729,7.706318,45.096732,30.767259,6.583651,96,30.767259,298.392040,NaN
0,5,83471233,1500,building,,apartments,56.362096,182.904191,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,92.202407,7.641764,2080,92.202407,269.298216,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,10217,1214584772,1500,building,,residential,17.299046,110.479065,7.698073,45.106321,7.698279,45.106267,7.698176,45.106294,21.066739,6.364065,111,21.066739,270.587673,1.0
0,10218,1233637785,1500,building,,,11.122694,262.512272,7.655736,45.134323,7.655596,45.134310,7.655666,45.134316,81.772403,44.343549,3026,81.772403,179.260131,NaN
1,10219,1233637785,1500,building,,,7.779111,352.529588,7.655596,45.134310,7.655583,45.134379,7.655589,45.134344,-87.541932,40.459978,464,272.458068,279.928481,1.0
2,10220,1233637785,1500,building,,,11.122699,82.511550,7.655583,45.134379,7.655723,45.134392,7.655653,45.134386,85.625732,44.459302,3026,85.625732,3.114182,NaN


In [ ]:
# Add column 'street_facing_manual' and, as default, set values as NaN.
# Upon importing the table into GIS, the building split lines should be manually reviewed at each threshold
# using symbology to view values of 1 and 0 in different colors.
# Then the best threshold can be copied into 'st_facing_manual' and the user can modify values of 1 or 0
# as required to reach the desired classification of street-facing segments.

prepped_splitlines['st_facing_manual'] = np.nan
prepped_splitlines.rename(columns = {'OBJECTID':'Orig_ObjID'}, inplace = True)
prepped_splitlines

,Orig_ObjID,osm_id,code,fclass,name,type,LINE_LENGTH,LINE_BEARING,Start_Lon,Start_Lat,End_Lon,End_Lat,Center_Lon,Center_Lat,osmRoads_NEAR_ANGLE,osmRoads_NEAR_DIST,osmRoads_NEAR_FID,osm_Roads_NEAR_ANGLE_corrected,angle_diff,st_facing_mult_1pt0,st_facing_manual
0,1,83471230,1500,building,,apartments,62.646302,182.411524,7.706395,45.096729,7.706361,45.096166,7.706378,45.096448,-86.363996,20.390748,1202,273.636004,91.224480,0.0,NaN
1,2,83471230,1500,building,,apartments,12.109261,272.427701,7.706361,45.096166,7.706208,45.096171,7.706285,45.096168,-79.177687,15.041523,1202,280.822313,8.394612,NaN,NaN
2,3,83471230,1500,building,,apartments,62.635242,2.411950,7.706208,45.096171,7.706241,45.096734,7.706225,45.096452,-86.364105,8.284624,1202,273.635895,271.223945,1.0,NaN
3,4,83471230,1500,building,,apartments,12.108680,92.375219,7.706241,45.096734,7.706395,45.096729,7.706318,45.096732,30.767259,6.583651,96,30.767259,298.392040,NaN,NaN
0,5,83471233,1500,building,,apartments,56.362096,182.904191,7.705838,45.097673,7.705801,45.097166,7.705819,45.097419,92.202407,7.641764,2080,92.202407,269.298216,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,10217,1214584772,1500,building,,residential,17.299046,110.479065,7.698073,45.106321,7.698279,45.106267,7.698176,45.106294,21.066739,6.364065,111,21.066739,270.587673,1.0,NaN
0,10218,1233637785,1500,building,,,11.122694,262.512272,7.655736,45.134323,7.655596,45.134310,7.655666,45.134316,81.772403,44.343549,3026,81.772403,179.260131,NaN,NaN
1,10219,1233637785,1500,building,,,7.779111,352.529588,7.655596,45.134310,7.655583,45.134379,7.655589,45.134344,-87.541932,40.459978,464,272.458068,279.928481,1.0,NaN
2,10220,1233637785,1500,building,,,11.122699,82.511550,7.655583,45.134379,7.655723,45.134392,7.655653,45.134386,85.625732,44.459302,3026,85.625732,3.114182,NaN,NaN


# Next steps in GIS

In [ ]:
# Export prepped_splitlines, to be re-imported into GIS.

folder_name = # ADD FOLDER NAME
file_name = # ADD FILE NAME + '.xlsx'
prepped_splitlines.to_excel(folder_name + file_name, index=False)

In [ ]:
'''
NEXT STEPS IN GIS:

1.  Import prepped_splitlines and join using 'Orig_ObjID' to the non-prepped split lines layer on its 'OBJECTID'.
    Note: the columns to be joined are all multipliers of 'st_facing_mult_', 'st_facing_manual', and 'angle_diff'.
2.  To check which 'st_facing_mult' muiltiplier works best for the case study, change GIS symbology with unique values to visualize the segments classified
    as street-facing. Once the best is identified, copy those values to 'st_facing_manual', which is manually modified with 1 or 0 as required to manually
    correct street-facing segments as required.
    Note: it is suggested to review street-facing lines with a web browser open with Google Maps displaying the layer 'Street View', which
    highlights all roads with GSV availability. Some roads that exist in GSV may not exist in the OSM roads layer, and vice versa.
3.  Once street-facing segments have been manually verified, use Python/Pandas to assign a sequential suffix to each line segment,
    i.e. _1, _2, _3, and so on. Store this in a new column osm_id_final, as this is required for other scripts in the package.

'''